In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay
import lightgbm as lgb
import matplotlib.pyplot as plt

# Tải dữ liệu
df = pd.read_csv('../data/processed/model_ready_data.csv')

print("Dữ liệu đã sẵn sàng cho mô hình:")
display(df.head())
df.info()

In [ ]:
# 1. Tách Features (X) và Target (y)
X = df.drop('review_score', axis=1)
y = df['review_score']

# 2. Phân loại các cột
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()

print(f"Numeric features: {numeric_features}")
print(f"Categorical features: {categorical_features}")

# 3. Xây dựng pipeline tiền xử lý (preprocessor)
#    - Đối với các cột số: Chuẩn hóa dữ liệu (scaling).
#    - Đối với các cột category: Chuyển đổi thành các cột nhị phân (One-Hot Encoding).
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# 4. Chọn mô hình
# LightGBM là một lựa chọn mạnh và nhanh cho dữ liệu dạng bảng
model = lgb.LGBMClassifier(random_state=42)

# 5. Kết hợp preprocessor và model thành một pipeline hoàn chỉnh
# Điều này đảm bảo rằng dữ liệu test cũng sẽ được xử lý y hệt dữ liệu train
full_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', model)
])

print("\nPipeline đã được xây dựng thành công!")

In [ ]:
# 1. Chia dữ liệu thành tập train và test (80% train, 20% test)
# `stratify=y` rất quan trọng để đảm bảo tỷ lệ các lớp review_score trong tập train và test là như nhau
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Kích thước tập train: {X_train.shape}")
print(f"Kích thước tập test: {X_test.shape}")

# 2. Huấn luyện pipeline
print("\nBắt đầu huấn luyện mô hình...")
full_pipeline.fit(X_train, y_train)
print("Huấn luyện hoàn tất!")

# 3. Đưa ra dự đoán trên tập test
y_pred = full_pipeline.predict(X_test)

# 4. Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"\nĐộ chính xác (Accuracy): {accuracy:.4f}")
print("\nBáo cáo phân loại (Classification Report):")
print(report)

# 5. Trực quan hóa ma trận nhầm lẫn (Confusion Matrix)
print("\nMa trận nhầm lẫn:")
fig, ax = plt.subplots(figsize=(8, 8))
ConfusionMatrixDisplay.from_estimator(full_pipeline, X_test, y_test,
                                      cmap=plt.cm.Blues,
                                      ax=ax,
                                      normalize='true') # Normalize để xem tỷ lệ %
plt.title('Ma trận nhầm lẫn (Normalised)')
plt.show()

In [ ]:
# Tạo thư mục models nếu chưa có
os.makedirs('../models', exist_ok=True)

# Lưu pipeline vào file
model_path = '../models/lgbm_review_score_predictor.joblib'
joblib.dump(full_pipeline, model_path)

print(f"\nĐã lưu mô hình đã huấn luyện tại: {model_path}")